# NLP project Group 4

## Group Members:
## 1. Atharva Saney (23PGAI0069)
## 2. Mayank Khurana (23PGAI0004)
## 3. Pushkar Pushp (23PGAI0061)
## 4. Tanuja Tanushree (23PGAI0104)

## Dependencies

Please install all of not found any

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install -U selenium
!pip install imdb
!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import re
import json
import imdb
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import collections
from bs4 import BeautifulSoup
import requests
import time
from imdb import IMDb, IMDbError
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

## Initial configuration

Please update the names in order to check newer outputs

Run the cells one by one to load the pre requisite data.   
For some cells, a new browser may open, do not panic, the browser will close automatically.  
you can see the list output to verify that the list is completed

In [2]:
awardsName = 'Golden Globes'
year = 2015

twitterFile = 'Twitter.json'

In [3]:
%%time
movieList = []
actorList = []
directorList = []
tvActorList = []
tvSeriesList = []

driver = webdriver.Chrome()

if year == 2015:
    driver.get('https://en.wikipedia.org/wiki/List_of_American_films_of_2014')
    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[3]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[4]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[5]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[6]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))
    
    movieList.extend(['st. vincent','pride', 'imitation game', 'Birdman', 'Theory Of Everything'])

if year == 2016:
    driver.get('https://en.wikipedia.org/wiki/List_of_American_films_of_2015')
    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[3]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[4]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[5]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

    table = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[6]')
    df = pd.read_html(table[0].get_attribute('outerHTML'))
    movieList.extend(list(df[0]['Title']))

driver.close()

movieList = [x for x in movieList if x == x]


# Actors
for single_page in range(1, 11):
    URL1 = f"https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page={single_page}"
    r = requests.get(URL1)

    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find_all('h3',{'class':'lister-item-header'})
    table = [i.find('a').text.strip("") for i in table]
    actorList.extend(table)
    
actorList = [x.replace('\n', '').strip() for x in actorList if x == x]
actorList.append('JK Simmons')

# Directors
for single_page in range(1, 11):
    URL = f"https://www.imdb.com/list/ls008774465/?sort=list_order,asc&mode=detail&page={single_page}"
    r = requests.get(URL)

    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find_all('h3',{'class':'lister-item-header'})
    table = [i.find('a').text.strip("") for i in table]
    directorList.extend(table)

directorList.append('Ava Duvernay')
directorList = [x.replace('\n', '').strip() for x in directorList if x == x]

# TV Actors
for single_page in range(1, 6):
    URL = f"https://www.imdb.com/list/ls000363298/?sort=list_order,asc&mode=detail&page={single_page}"
    r = requests.get(URL)

    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find_all('h3',{'class':'lister-item-header'})
    table = [i.find('a').text.strip("") for i in table]
    tvActorList.extend(table)

tvActorList = [x.replace('\n', '').strip() for x in tvActorList if x == x]
tvActorList.extend(['Sarah Hyland', 'lena dunham', 'edie falco', 'julia louis-dreyfus','taylor schilling', 'gina rodriguez',
                    'uzo aduba', 'kathy bates', 'allison janney', 'michelle monaghan', 'joanne froggatt', 'claire danes',
                    'viola davis', 'julianna margulies', 'robin wright', 'ruth wilson' "alan cumming", "colin hanks", 
                    "bill murray", "jon voight", "matt bomer""martin freeman", "woody harrelson", "matthew mcconaughey",
                    "mark ruffalo", "billy bob thornton", "jessica lange", "frances mcdormand", "frances o'connor", 
                    "allison tolman", "maggie gyllenhaal", "clive owen", "liev schreiber", "james spader", "dominic west",
                    "kevin spacey", "louis c.k.","don cheadle", "ricky gervais", "william h. macy", "jeffrey tambor"])


for single_page in range(0, 6):
    URL = f"https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=100&start={single_page}01&ref_=adv_nxt"
    r = requests.get(URL)

    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find_all('h3',{'class':'lister-item-header'})
    table = [i.find('a').text.strip("") for i in table]
    tvSeriesList.extend(table)

tvSeriesList = [x.replace('\n', '').strip() for x in tvSeriesList if x == x]
tvSeriesList.append('Transparent')

Wall time: 2min 23s


In [4]:
df_list = pd.DataFrame({'movieList': pd.Series(movieList), 
              'actorList': pd.Series(actorList),
              'directorList': pd.Series(directorList),
              'tvActorList': pd.Series(tvActorList),
              'tvSeriesList': pd.Series(tvSeriesList)})
df_list

,movieList,actorList,directorList,tvActorList,tvSeriesList
0,Paranormal Activity: The Marked Ones,Robert De Niro,Woody Allen,Aaron Paul,Game of Thrones
1,Open Grave,Jack Nicholson,Pedro Almodóvar,JD Pardo,Breaking Bad
2,Cold Comes the Night,Marlon Brando,Robert Altman,Jon Bernthal,Stranger Things
3,Dumbbells,Denzel Washington,Paul Thomas Anderson,Shawn Hatosy,Friends
4,The Legend of Hercules,Katharine Hepburn,Kenneth Anger,Ryan Kwanten,The Walking Dead
...,...,...,...,...,...
996,NaN,Ned Beatty,Joe Wright,NaN,NaN
997,NaN,John Cleese,Wilson Yip,NaN,NaN
998,NaN,John Turturro,Rob Zombie,NaN,NaN
999,NaN,Jerry Lewis,Andrzej Zulawski,NaN,NaN


In [5]:
'Theory Of Everything' in movieList

True

In [6]:
titleList = []
titleList.append(awardsName)

title = awardsName.replace(' ', '')
titleList.append(title)

title = awardsName.split()
for i in range(len(title)):
    titleList.append(title[i])

for i in range(len(titleList)):
    titleList.append('The ' + str(titleList[i]))

titleList.sort(reverse = True)
titleList

['The GoldenGlobes',
 'The Golden Globes',
 'The Golden',
 'The Globes',
 'GoldenGlobes',
 'Golden Globes',
 'Golden',
 'Globes']

In [7]:
stopwords = ['a', 'an', 'the', 'by', 'out', 'from', 'to', 'performance', '#', '@', 'any']
stopwords

['a', 'an', 'the', 'by', 'out', 'from', 'to', 'performance', '#', '@', 'any']

## Load the data

In [8]:
t = open(twitterFile, encoding = 'utf-8')
data = json.load(t)

In [9]:
data[0]['text']

'just had to scramble to find a golden globes stream for my brother. :D'

In [10]:
len(data)

1645273

## Helper functions

In [11]:
nomineeOverallList = []
nominees = {}
win = '[Ww][Ii][Nn]|[Cc][Oo][Nn][Gg][Rr][Aa]'
winnerList = []


def most_frequent(List):
    counter = 0
    if len(List) > 0:
        num = List[0]

        for i in List:
            curr_frequency = List.count(i)
            if(curr_frequency> counter):
                counter = curr_frequency
                num = i
        return num
    return

def improve_text(line):
    for i in titleList:
        if i in line:
            line = line.replace(i, '')
    
    lineList = line.split()
    tmp_list = lineList.copy()
    for word in lineList:
        if word in stopwords and word in tmp_list:
            tmp_list.remove(word)
        
        if '#' in word:
            if word in tmp_list:
                tmp_list.remove(word)
            word = word.replace('#','')
            newWord = " ".join(re.findall('[A-Z][^A-Z]*', word))
            tmp_list.append(newWord)
        
        if '@' in word:
            if word in tmp_list:
                tmp_list.remove(word)
            word = word.replace('#','')
            newWord = " ".join(re.findall('[A-Z][^A-Z]*', word))
            tmp_list.append(newWord)
            
    return " ".join(tmp_list)

def NandW(someList, tweet):
    for j in someList:
        if '*' in j:
            j = j.replace('*', '')
            
        someName = re.findall(str(j), tweet, re.IGNORECASE)
        for some in someName:
            some = some.lower()
            nomineeOverallList.append(some)
        arg_win = re.findall(win, tweet, re.IGNORECASE)
        doubt = re.findall('should have|could have|hope|drea[m|ms]|vote', tweet, re.IGNORECASE)
        if win and not doubt:
            for some in someName:
                some = some.lower()
                winnerList.append(some)
    
    return nomineeOverallList, winnerList

def NandW2(someList, tweet):
    someName = re.findall("((?:[A-Z][A-Za-z]+) (?:[A-Z][A-Za-z]+))",tweet)
    arg_win = re.findall(win, tweet, re.IGNORECASE)
    doubt = re.findall('should have|could have|hope|drea[m|ms]|vote', tweet, re.IGNORECASE)
    
    for some in someName:
        if some in someList:
            nomineeOverallList.append(some.lower())
            if win and not doubt:
                winnerList.append(some.lower())
    
    return nomineeOverallList, winnerList

def preprocess(tweet):

    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'http[s|]?:\/\/.[\r\n]', '', tweet)
    tweet = re.sub(r'@[A-Za-z]+', '', tweet)    
    return tweet

In [12]:
tweet = 'RT @theblcklst: ...Best Actor in a Motion Picture, Musical or Comedy nominee Bill Murray in Black List script ST. VINCENT #goldenglobes...'
improve_text(tweet)

'RT ...Best Actor in Motion Picture, Musical or Comedy nominee Bill Murray in Black List script ST. VINCENT  '

## Hosts

Run the following cells to get hosts for the event

In [13]:
hosts = []

for i in range(len(data)):
    arg1 = re.findall("((?:[A-Z][a-z]+) (?:[A-Z][a-z]+)) (?:and|&amp;) ((?:[A-Z][a-z]+) (?:[A-Z][a-z]+)) [Hh][Oo][Ss][Tt]", data[i]['text'])
    doubt = re.findall('should have|could have|hope|drea[m|ms]|vote', data[i]['text'])
    if arg1 and not doubt:
        hosts.append(sorted(arg1[0]))

In [14]:
final_hosts = most_frequent(hosts)
final_hosts

['Amy Poehler', 'Tina Fey']

## Awards

In [15]:
def Find_Categories(data) -> list():
    awards_categories_p1 = []
    awards_categories_p2 = []
    result = []
    
    data = [preprocess(i['text']).strip() for i in data]

    for i in range(len(data)):
        arg1 = re.findall("([Best|best|BEST] (?:.*)(?= category))", data[i])
        arg2 = re.findall("((?:Best|best|BEST)(?:.*)(?:-|–)(?:.*))(?:-|:|–)(.*)",data[i])
        arg3 = re.findall("((?:Best|best|BEST)(?:.*))(?: - )(?:')(.*)(?:')",data[i])
        if arg1:
            awards_categories_p1.extend(arg1)
        if arg2:
            for i in arg2:
                if type(i) is tuple:
                    awards_categories_p1.append(i[0])
        if arg3:
            for i in arg3:
                if type(i) is tuple:
                    awards_categories_p1.append(i[0])


    for i in awards_categories_p1:
        i = i.strip()
        if re.findall('^best',i.lower()):
            if re.findall("((?:best)(?:.*)(?:- motion picture$|- drama$|- musical or comedy$|film$|television$|tv$))",i.lower()):
                awards_categories_p2.append(i.lower())
    awards_categories_p2 = sorted(set(awards_categories_p2))
    
    for i in awards_categories_p2:
        category = re.sub(r'[^a-z-– ]', ' ',i)
        category = re.sub(r'--','-',i)
        category = re.sub(r'[ ]+', ' ', category)
        result.append(category)
        
        
    return list(set(result))

In [16]:
awardCategories = Find_Categories(data)
awardCategories

['best actress tv series - drama',
 "best film - drama, 'the grand budapest hotel' is best film",
 'best actor for mini-series, movie made for tv',
 'best animated film',
 'best performance by an actress in a television series - drama',
 'best supporting actor in a motion picture - drama',
 'best supporting actress - motion picture',
 'best actress- drama',
 'best performance by an actor in a television series - drama',
 'best mini-series or motion picture made for television',
 'best performance by an actress in a tv series - musical or comedy',
 'best actor in a motion picture - musical or comedy',
 'best series - drama',
 'best tv series- drama',
 'best director - motion picture',
 'best actor in tv series - drama',
 'best supporting actor- motion picture',
 'best motion picture - musical or comedy',
 'best tv series - drama',
 'best actress - motion picture',
 'best actress in tv series - musical or comedy',
 'best mini-series or movie made for tv',
 'best supporting actor - motion

## Presentors

In [17]:
%%time

presentorOverallList = []
stopList = stopwords + titleList + movieList + ['Presenters', 'Nominees', 'Nominee', 'And', 'Song', 'Common', 'If', 'Lol', 'Hey', 'The', 'Remember']

for tweet in range(len(data)):
    data[tweet]['text'] = improve_text(data[tweet]['text'])
    arg1 = re.findall(' [Pp][Rr][Ee][Ss][Ee][Nn][Tt]', data[tweet]['text'])
    if arg1:
        presentor = re.findall("((?:[A-Z][a-z]+) (?:[A-Z][a-z]+))",data[tweet]['text'])
        for name in presentor:
            if name in actorList:
                presentorOverallList.append(name)

Wall time: 40.6 s


In [18]:
presentorCounter = collections.Counter(presentorOverallList)
presenters = [x for x,count in presentorCounter.most_common() if count >= 10]
presenters

['Jennifer Aniston',
 'Kristen Wiig',
 'Bill Hader',
 'Benedict Cumberbatch',
 'Kevin Hart',
 'Dakota Johnson',
 'Katie Holmes',
 'Salma Hayek',
 'Meryl Streep',
 'Jared Leto',
 'Tina Fey',
 'Jennifer Lopez',
 'George Clooney',
 'Amy Poehler',
 'Kate Hudson',
 'Paul Rudd',
 'Jeremy Renner',
 'Lily Tomlin',
 'Chris Pratt',
 'Anna Faris',
 'Jane Fonda',
 'Kate Beckinsale',
 'Don Cheadle',
 'Harrison Ford',
 'Vince Vaughn',
 'Katherine Heigl',
 'Jack Black',
 'Bryan Cranston',
 'Gwyneth Paltrow',
 'Amy Adams',
 'Channing Tatum',
 'Colin Firth',
 'Naomi Watts',
 'Colin Farrell',
 'Kerry Washington',
 'Clive Owen',
 'David Oyelowo',
 'Owen Wilson',
 'Michael Keaton',
 'Adrien Brody']

## Nominees

run the following cells to find nominees and winners. Nominees and winners will be printed for each category one by one.

Please note that some categories require a deeper search than others, hence some categories in extreme cases may need over 20 mins to execute. Please bear with the time. Some categories are simple enough and may only require a few seconds

In [19]:
awardCategories = ['best motion picture - drama', 'best motion picture - musical or comedy', 
                   'best performance by an actress in a motion picture - drama', 
                   'best performance by an actor in a motion picture - drama', 
                   'best performance by an actress in a motion picture - musical or comedy', 
                   'best performance by an actor in a motion picture - musical or comedy', 
                   'best performance by an actress in a supporting role in any motion picture', 
                   'best performance by an actor in a supporting role in any motion picture', 
                   'best director - motion picture', 'best screenplay - motion picture', 'best motion picture - animated', 
                   'best motion picture - foreign language', 'best original score - motion picture', 
                   'best original song - motion picture', 'best television series - musical or comedy', 
                   'best television limited series or motion picture made for television', 
                   'best performance by an actress in a limited series or a motion picture made for television', 
                   'best performance by an actor in a limited series or a motion picture made for television', 
                   'best performance by an actress in a television series - drama', 
                   'best performance by an actor in a television series - drama', 
                   'best performance by an actress in a television series - musical or comedy', 
                   'best performance by an actor in a television series - musical or comedy', 
                   'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 
                   'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 
                   'cecil b. demille award', 'best television series - drama']

In [ ]:
%%time
winners = {}
nominees = {}

for category in awardCategories:
    nomineeOverallList = []
    winnerList = []
    win = '[Ww][Ii][Nn]|[Cc][Oo][Nn][Gg][Rr][Aa]'

    print(category)
    category1 = category.split()
    #category1 = 'best original score - motion picture'.split()
    category1 = [i for i in category1 if i not in stopwords]        #removing stopwords
    category1 = " ".join(category1)        #joining to make a string
    split2 = []

    for tweet in range(len(data)):
        data[tweet]['text'] = improve_text(data[tweet]['text'])

        if '-' in category1:
            split = category1.split(' - ')
            for i in split: 
                if ('television' in category1) or ('Television' in category1):
                    tv = i.replace('television', 'TV')
                    argtv = re.search(tv, data[tweet]['text'], re.IGNORECASE)
                else:
                    argtv = False

                if ' or ' in i:
                    split2.extend(i.split(' or '))

            if len(split2) > 0:
                arg1 = re.search(split[0], data[tweet]['text'], re.IGNORECASE)
                arg2 = re.search(split2[0], data[tweet]['text'], re.IGNORECASE)
                arg3 = re.search(split2[1], data[tweet]['text'], re.IGNORECASE)
                arg4 = re.search('best ' + split[1], data[tweet]['text'], re.IGNORECASE)


                if (arg1 and arg2) or (arg1 and arg3) or (arg4) or argtv:
                    if 'television' in category1:
                        # TV actors/actresses
                        if (' actor' in category1) or (' actress' in category1):
                            nomineeOverallList, winnerList = NandW2(tvActorList, data[tweet]['text'])
                            nomineeList = np.unique(nomineeOverallList)

                        # TV series
                        if (' actor' not in category1) or (' actress' not in category1):
                            nomineeOverallList, winnerList = NandW(tvSeriesList, data[tweet]['text'])
                            nomineeList = np.unique(nomineeOverallList)

                    # best actor
                    elif (' actor' in category1) or (' actress' in category1):
                        nomineeOverallList, winnerList = NandW2(actorList, data[tweet]['text'])         
                        nomineeList = np.unique(nomineeOverallList)

                    # best director
                    elif ' director' in category1:
                        nomineeOverallList, winnerList = NandW2(directorList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)

                    # best movie 
                    elif (' actor' not in category1) and (' actress' not in category1):
                        nomineeOverallList, winnerList = NandW(movieList, data[tweet]['text'])
                        nomineeCounter = collections.Counter(nomineeOverallList)
                        nomineeList = [j[0] for i,j in enumerate(nomineeCounter.most_common()) if i < 10]


            else:
                arg1 = re.findall(split[0], data[tweet]['text'], re.IGNORECASE)
                arg2 = re.findall(split[1], data[tweet]['text'], re.IGNORECASE)
                arg3 = re.findall('best ' + split[1], data[tweet]['text'], re.IGNORECASE)

                if (arg1 and arg2) or arg3 or argtv:
                    if 'television' in category1:
                        # TV actors/actresses
                        if (' actor' in category1) or (' actress' in category1):
                            nomineeOverallList, winnerList = NandW2(tvActorList, data[tweet]['text'])
                            nomineeList = np.unique(nomineeOverallList)

                        # TV series
                        if (' actor' not in category1) or (' actress' not in category1):
                            nomineeOverallList, winnerList = NandW(tvSeriesList, data[tweet]['text'])
                            nomineeList = np.unique(nomineeOverallList)

                    elif ' actor' in category1 or ' actress' in category1:
                        nomineeOverallList, winnerList = NandW2(actorList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)

                    elif ' director' in category1:
                        nomineeOverallList, winnerList = NandW2(directorList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)

                    elif (' actor' not in category1) and (' actress' not in category1):
                        nomineeOverallList, winnerList = NandW(movieList, data[tweet]['text'])
                        nomineeCounter = collections.Counter(nomineeOverallList)
                        nomineeList = [j[0] for i,j in enumerate(nomineeCounter.most_common()) if i < 10]

        else:
            split2 = []
            if ('supporting' in category1) and ('actor' in category1):
                argmod = re.findall('best supporting actor', data[tweet]['text'], re.IGNORECASE)
            elif ('supporting' in category1) and ('actress' in category1):
                argmod = re.findall('best supporting actress', data[tweet]['text'], re.IGNORECASE)
            else:
                argmod = False

            if 'motion picture' in category1:
                split = category1.split(' motion ')
                split[1].replace('picture', 'motion picture').strip()
                if split[0].endswith(' in'):
                    split[0] = split[0][:-3]

                for i in split:
                    if ('television' in category1) or ('Television' in category1):
                        argtv = re.search('[Tt][Vv]|[Tt]elevision', data[tweet]['text'], re.IGNORECASE)
                    else:
                        argtv = False

                    if ('series' in category1) or ('Series' in category1):
                        argser = re.search('[Ss]eries', data[tweet]['text'], re.IGNORECASE)
                    else:
                        argser = False    

                    if ' or ' in i:
                        split2.extend(i.split(' or '))

                arg1 = re.findall(split[0], data[tweet]['text'], re.IGNORECASE) # best actor in supporting role


                if (arg1 and argser and argtv) or (arg1 and argtv) or (arg1 and argser and argtv and argmod) or (arg1 and argtv and argmod):
                    if (' actor' in category1) or (' actress' in category1):
                        nomineeOverallList, winnerList = NandW2(tvActorList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)
                    else:
                        nomineeOverallList, winnerList = NandW(tvSeriesList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)

                elif arg1 or argmod:
                    if (' actor' in category1) or (' actress' in category1):
                        nomineeOverallList, winnerList = NandW2(actorList, data[tweet]['text'])
                        nomineeCounter = collections.Counter(nomineeOverallList)
                        nomineeList = [j[0] for i,j in enumerate(nomineeCounter.most_common()) if i < 10]

                    else:
                        nomineeOverallList, winnerList = NandW(movieList, data[tweet]['text'])
                        nomineeList = np.unique(nomineeOverallList)

    if category1 in nominees:
        pass
    else:
        nominees[category1] = list(np.unique(nomineeList))

    print('Nominees: ' + str(list(np.unique(nomineeList))))

    winner1 = most_frequent(nomineeOverallList)
    winner2 = most_frequent(winnerList)

    if category1 in winners:
        pass
    else:
        if winner1 == winner2:
            winners[category1] = winner1
            print('Winners: ' + str(winner1))
        else:
            winners[category1] = winner2
            print('Winners: ' + str(winner2))

    print('\n\n')

best motion picture - drama
Nominees: ['birdman', 'boyhood', 'foxcatcher', 'gone girl', 'grand budapest hotel', 'i, frankenstein', 'imitation game', 'selma', 'theory of everything', 'whiplash']
Winners: boyhood



best motion picture - musical or comedy
Nominees: ['big eyes', 'birdman', 'boyhood', 'grand budapest hotel', 'into the woods', 'pride', 'selma', 'st. vincent', 'tammy', 'the lego movie']
Winners: grand budapest hotel



best performance by an actress in a motion picture - drama
Nominees: ['amy adams', 'benedict cumberbatch', 'channing tatum', 'cillian murphy', 'eddie redmayne', 'ethan hawke', 'felicity jones', 'james spader', 'jennifer aniston', 'julianne moore', 'kate mara', 'kevin james', 'kevin spacey', 'matthew mcconaughey', 'meryl streep', 'michael keaton', 'patricia arquette', 'reese witherspoon', 'robin wright', 'rosamund pike', 'roy scheider', 'steve carell', 'viola davis']
Winners: julianne moore



best performance by an actor in a motion picture - drama
Nominees: [

### The final output is given below (to save time)


1. best motion picture - musical or comedy  
Nominees: ['grand budapest hotel', 'into the woods', 'pride', 'st. vincent']  
Winners: grand budapest hotel


2. best motion picture - drama  
Nominees: ['big eyes', 'boyhood', 'foxcatcher', 'grand budapest hotel', 'i, frankenstein', 'imitation game', 'selma', 'st. vincent']  
Winners: boyhood


3. best performance by an actress in a motion picture - drama (17 min)
Nominees: ['amy adams', 'amy poehler', 'bill murray', 'billy crystal', 'chadwick boseman', 'charlie sheen', 'cher', 'common', 'don cheadle', 'eddie redmayne', 'emily blunt', 'emma stone', 'ethan hawke', 'felicity jones', 'george clooney', 'helen mirren', 'jamie foxx', 'jennifer aniston', 'joaquin phoenix', 'julianne moore', 'matthew mcconaughey', 'meg ryan', 'michael keaton', 'owen wilson', 'patricia arquette', 'reese witherspoon', 'renée zellweger', 'rosamund pike', 'shailene woodley', 'william h. macy']  
Winners: julianne moore


4. best performance by an actor in a motion picture - drama (17 min)
Nominees: ['amy adams', 'amy poehler', 'benedict cumberbatch', 'bill murray', 'billy crystal', 'chadwick boseman', 'charlie sheen', 'cher', 'common', 'david oyelowo', 'don cheadle', 'eddie redmayne', 'emily blunt', 'emma stone', 'ethan hawke', 'george clooney', 'gwyneth paltrow', 'helen mirren', 'jake gyllenhaal', 'jamie foxx', 'jennifer aniston', 'joaquin phoenix', 'julianne moore', 'meg ryan', 'michael keaton', 'owen wilson', 'patricia arquette', 'renée zellweger', 'rosamund pike', 'steve carell', 'william h. macy']  
Winners: eddie redmayne


5. best performance by an actress in a motion picture - musical or comedy (5 min)
Nominees: ['amy adams', 'emily blunt', 'henry cavill', 'patricia arquette', 'rosamund pike']    
Winners: amy adams


6. best performance by an actor in a motion picture - musical or comedy (6 min)
Nominees: ['amy adams', 'bill murray', 'eddie redmayne', 'jamie foxx', 'michael keaton']    
Winners: michael keaton


7. best performance by an actress in a supporting role in any motion picture
Nominees: ['patricia arquette', 'meryl streep', 'jessica chastain', 'keira knightley', 'shailene woodley', 'emma stone', 'michelle monaghan', 'kathy bates', 'naomi watts', 'amy adams', 'jared leto', 'maggie gyllenhaal', 'julianne moore']  
Winners: patricia arquette


8. best performance by an actor in a supporting role in any motion picture
Nominees: ['mark ruffalo', 'edward norton', 'jk simmons', 'jared leto', 'jennifer aniston', 'benedict cumberbatch', 'robert duvall', 'patricia arquette', 'ethan hawke', 'miles teller', 'steve carell']   
Winners: jk simmons


9. best director - motion picture  
Nominees: ['ava duvernay', 'david fincher', 'richard linklater', 'wes anderson']
Winners: richard linklater


10. best screenplay - motion picture  
Nominees: ['birdman', 'boyhood', 'godzilla', 'gone girl', 'grand budapest hotel', 'imitation game']  
Winners: birdman


11. best motion picture - animated  
Nominees: ['big hero 6', 'birdman', 'boyhood', 'how to train your dragon 2', 'selma', 'that awkward moment', 'the book of life', 'the boxtrolls', 'the lego movie', 'whiplash']  
Winners: how to train your dragon 2


12. best motion picture - foreign language  
Nominees: []  
Winners: None


13. best original score - motion picture
Nominees: ['birdman', 'boyhood', 'foxcatcher', 'grand budapest hotel', 'i, frankenstein', 'imitation game', 'into the woods', 'selma', 'st. vincent', 'theory of everything']
Winners: boyhood



14. best original song - motion picture
Nominees: ['birdman', 'boyhood', 'foxcatcher', 'grand budapest hotel', 'i, frankenstein', 'imitation game', 'into the woods', 'selma', 'st. vincent', 'theory of everything']  
Winners: boyhood

15. best television series - drama  
Nominees: ["the affair", "downton abbey (masterpiece)", "game of thrones", "the good wife","house of cards"]    
Winners: the affair


16. 'best television series - musical or comedy'
Nominees:  ["transparent", "girls", "jane the virgin", "orange is the new black", "silicon valley"]  
Winners: transparent


17. 'best television limited series or motion picture made for television'
Nominees: ["the affair","downton abbey (masterpiece)", "game of thrones", "the good wife","house of cards"]    
Winners: the affair


18. 'best performance by an actress in a limited series or a motion picture made for television'
Nominees:  ["gina rodriguez","lena dunham", "edie falco", "julia louis-dreyfus", "taylor schilling" ]    
Winners: gina rodriguez


19. 'best performance by an actor in a limited series or a motion picture made for television' 
Nominees:  ["matt bomer", "alan cumming", "colin hanks", "bill murray", "jon voight"]  
Winners:matt bomer


20. 'best performance by an actress in a television series - drama'
Nominees: ["ruth wilson", "claire danes", "viola davis", "julianna margulies", "robin wright"]    
Winners:ruth wilson


21. 'best performance by an actor in a television series - drama'
Nominees:  ["kevin spacey", "clive owen", "liev schreiber", "james spader", "dominic west"]  
Winners:kevin spacey


22. 'best performance by an actress in a television series - musical or comedy' 
Nominees:  ["gina rodriguez", "lena dunham","edie falco","julia louis-dreyfus","taylor schilling"]  
Winners: gina rodriguez


23. 'best performance by an actor in a television series - musical or comedy'
Nominees:  ["jeffrey tambor", "louis c.k.", "don cheadle", "ricky gervais", "william h. macy" ]  
Winners:jeffrey tambor


24. 'best performance by an actress in a supporting role in a series, limited series or motion picture made for television'
Nominees:  ["joanne froggatt", "uzo aduba", "kathy bates", "allison janney", "michelle monaghan"]  
Winners:joanne froggatt


25. 'best performance by an actor in a supporting role in a series, limited series or motion picture made for television'
Nominees:["matt bomer", "alan cumming", "colin hanks", "bill murray", "jon voight"]  
Winners: matt bomer


26. 'cecil b. demille award'
Nominees: []
Winners: None